In [1]:
import copy
import datetime
import gc
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from scipy.io import loadmat, savemat
from shapely.geometry import Point
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

%matplotlib inline

## Preparation

Set seeds for reproducability

In [2]:
random_seed = 1
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)

Set device

In [3]:
gc.collect()
torch.cuda.empty_cache()
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device = torch.device('cpu')

## Load and preprocess data

Set paths to training data

In [4]:
dspl_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_dspl'
dsplRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_dsplRadial'
trac_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_trac'
tracRadial_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/train/trainData104/foo_tracRadial'

In [5]:
def data_to_npArrays(dspl_path, dsplRadial_path, trac_path, tracRadial_path):
    number_samples = len([name for name in os.listdir(dspl_path) if os.path.isfile(os.path.join(dspl_path, name))])
    number_radials = len([name for name in os.listdir(dsplRadial_path) if os.path.isfile(os.path.join(dsplRadial_path, name))])
    
    # save all samples in matrix
    samples = [] 
    for i, filename in enumerate(os.listdir(dspl_path)):
        f = os.path.join(dspl_path, filename)
        if os.path.isfile(f):
            sample = loadmat(f)
            if '__header__' in sample: del sample['__header__']
            if '__version__' in sample: del sample['__version__']
            if '__globals__' in sample: del sample['__globals__']
            sample['name'] = filename
            samples = np.append(samples, sample)
        else:
            continue
    samples = np.array(samples)

    # save all radial patterns of displacements in matrix
    dspl_radials = []
    for i, filename in enumerate(os.listdir(dsplRadial_path)):
        f = os.path.join(dsplRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            dspl_radials = np.append(dspl_radials, radial)
        else:
            continue
    dspl_radials = np.array(dspl_radials)
    
    # save all targets in matrix
    targets = []
    for i, filename in enumerate(os.listdir(trac_path)):
        f = os.path.join(trac_path, filename)
        if os.path.isfile(f):
            target = loadmat(f)
            if '__header__' in target: del target['__header__']
            if '__version__' in target: del target['__version__']
            if '__globals__' in target: del target['__globals__']
            target['name'] = filename
            targets = np.append(targets, target)
        else:
            continue 
    targets = np.array(targets)
    
    # save all radial patterns of traction forces in matrix
    trac_radials = []
    for i, filename in enumerate(os.listdir(tracRadial_path)):
        f = os.path.join(tracRadial_path, filename)
        if os.path.isfile(f):
            radial = loadmat(f)
            if '__header__' in radial: del radial['__header__']
            if '__version__' in radial: del radial['__version__']
            if '__globals__' in radial: del radial['__globals__']
            radial['name'] = filename
            trac_radials = np.append(trac_radials, radial)
        else:
            continue
    trac_radials = np.array(trac_radials)

    return samples, dspl_radials, targets, trac_radials

Create numpy arrays for samples and targets

In [6]:
samples, dspl_radials, targets, trac_radials = data_to_npArrays(dspl_path, dsplRadial_path, trac_path, tracRadial_path)

Split training data into train and validation set using stratified samples

In [7]:
radial_X_train, radial_X_val, radial_y_train, radial_y_val = train_test_split(dspl_radials, trac_radials, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(samples, targets, test_size=0.05, random_state=0)
X_train, X_val, y_train, y_val = np.append(radial_X_train, X_train), np.append(radial_X_val, X_val), np.append(radial_y_train, y_train), np.append(radial_y_val, y_val)

Extract displacement and traction fields from the training data and reshape to (samples, channels, depth, heigth, width)

In [8]:
X_train, X_val, y_train, y_val = np.array([sample['dspl'] for sample in X_train]), np.array([sample['dspl'] for sample in X_val]), np.array([target['trac'] for target in y_train]), np.array([target['trac'] for target in y_val])
# Reshape to (samples, channels, depth, height, width)
X_train = np.moveaxis(X_train[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
X_val = np.moveaxis(X_val[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
y_train = np.moveaxis(y_train[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
y_val = np.moveaxis(y_val[:, np.newaxis], [2, 3, 4], [-1, 3, 2])

Normalize training data

In [9]:
#x_min = X.min(axis=(3, 4), keepdims=True)
#x_max = X.max(axis=(3, 4), keepdims=True)
#X = (X - x_min)/(x_max-x_min)

#y_min = y.min(axis=(3, 4), keepdims=True)
#y_max = y.max(axis=(3, 4), keepdims=True)
#y = (y - y_min)/(y_max-y_min)

Convert train and validation data to Pytorch tensors

In [10]:
X_train = torch.from_numpy(X_train).double()
X_val = torch.from_numpy(X_val).double()
y_train = torch.from_numpy(y_train).double()
y_val = torch.from_numpy(y_val).double()

In [11]:
train_set = TensorDataset(X_train, y_train)
val_set = TensorDataset(X_val, y_val)

batch_size = 16

dataloaders = {}
dataloaders['train'] = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=3)
dataloaders['val'] = DataLoader(val_set, batch_size=3*batch_size, shuffle=True, num_workers=3)

## Model setup

In [12]:
NAME = "TracNet104-{:%Y-%b-%d %H:%M:%S}".format(datetime.datetime.now())
writer = SummaryWriter(log_dir='logs/{}'.format(NAME))

2022-05-04 14:14:05.861222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-04 14:14:05.861240: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [13]:
class ConvBlock_1(nn.Module):
    """Conv3D -> BatchNorm -> ReLU"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=(2,3,3), padding='same', bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_block_1(x)

In [14]:
class ConvBlock_2(nn.Module):
    """Conv3D -> ReLU"""
    
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv_block_2 = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=(2,3,3), padding='same'),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv_block_2(x)

In [15]:
class TracNet(nn.Module):
    def __init__(self, n_channels):
        super().__init__()
        
        self.s1 = ConvBlock_1(n_channels, 32)
        self.s2 = ConvBlock_2(32, 64)
        self.s3 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        self.s4 = ConvBlock_1(64, 64)
        self.s5 = ConvBlock_2(64, 128)
        self.s6 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        self.s7 = ConvBlock_1(128, 128)
        self.s8 = ConvBlock_2(128, 256)
        self.s9 = nn.MaxPool3d(kernel_size=(1, 2, 2))
        self.s10 = ConvBlock_1(256, 128)
        self.s11 = ConvBlock_1(128, 256)
        self.s12 = nn.ConvTranspose3d(256, 256, kernel_size=(1, 3, 3), stride=(1,2,2))
        #fusion3
        self.s13 = ConvBlock_1(512, 64)
        self.s14 = ConvBlock_1(64, 128)
        self.s15 = nn.ConvTranspose3d(128, 128, kernel_size=(1, 3, 3), stride=(1, 2, 2))
        #fusion2
        self.s16 = ConvBlock_1(256, 32)
        self.s17 = ConvBlock_1(32, 64)
        self.s18 = nn.ConvTranspose3d(64, 64, kernel_size=(1, 3, 3), stride=(1, 2, 2))
        #fusion1
        self.s19 = ConvBlock_1(128, 1)
        self.s20 = ConvBlock_1(1, 32)
        self.s21 = nn.Conv3d(32, 1, kernel_size=(2, 3, 3), padding='same')
        
    def forward(self, x):
        x1 = self.s1(x)
        x2 = self.s2(x1)
        x3 = self.s3(x2)
        x4 = self.s4(x3)
        x5 = self.s5(x4)
        x6 = self.s6(x5)
        x7 = self.s7(x6)
        x8 = self.s8(x7)
        x9 = self.s9(x8)
        x10 = self.s10(x9) 
        x11 = self.s11(x10)
        x12 = self.s12(x11)
        padded = torch.nn.functional.pad(x12, (0,-1,0,-1), 'constant', 0)
        fusion3 = torch.cat((x8, padded), dim=1)
        x13 = self.s13(fusion3)
        x14 = self.s14(x13)
        x15 = self.s15(x14)
        padded = torch.nn.functional.pad(x15, (0,-1,0,-1), 'constant', 0)
        fusion2 = torch.cat((x5, padded), dim=1)
        x16 = self.s16(fusion2)
        x17 = self.s17(x16)
        x18 = self.s18(x17)
        padded = torch.nn.functional.pad(x18, (0,-1,0,-1), 'constant', 0)
        fusion1 = torch.cat((x2, padded), dim=1)
        x19 = self.s19(fusion1)
        x20 = self.s20(x19)
        logits = self.s21(x20)
        return logits


Sample weights for convolutional layers from normal distribution

In [16]:
def initialize_weight(module):
    if isinstance(module, (nn.Conv3d, nn.ConvTranspose3d)):
        torch.nn.init.normal_(module.weight, std=0.01)

Define custom loss function corresponding to the forward loss function in the Matlab regression layer for image-to-image networks 
 
$${loss} = \frac{1}{2} \sum \limits _{p=1} ^{HWC} (t_{p} - y_{p})^{2}$$

In [17]:
class Custom_Loss(nn.Module):
    def __init__(self):
        super(Custom_Loss, self).__init__();
    
    def forward(self, predictions, target):
        loss = 0.5 * torch.sum(torch.pow(target - predictions, 2))
        return loss

In [18]:
def run_epoch(model, optimizer, dataloader, train):
    loss_fn = Custom_Loss()
    
    # Set model to training mode
    if train:
        model.train()
    else:
        model.eval()
    
    epoch_loss = 0.0
    epoch_rmse = 0.0
    
    # Iterate over data
    for xb, yb in dataloader:
        xb, yb = xb.to(device), yb.to(device)

        # zero the parameters
        if train:
            optimizer.zero_grad()

        # forward
        with torch.set_grad_enabled(train):
            pred = model(xb)
            loss = loss_fn(pred, yb)

            # backward + optimize if in training phase
            if train:
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type=2)                
                optimizer.step()

        # statistics
        epoch_loss += loss.item()
    
    epoch_loss /= len(dataloader.dataset)
    epoch_rmse = np.sqrt(2 * epoch_loss)
    return epoch_loss, epoch_rmse

In [19]:
def fit(model, optimizer, scheduler, dataloaders, max_epochs, patience):
    best_val_rmse = np.inf
    best_epoch = -1
    
    for epoch in range(1, max_epochs+1):
        train_loss, train_rmse = run_epoch(model, optimizer, dataloaders['train'], train=True)
        scheduler.step()
        val_loss, val_rmse = run_epoch(model, None, dataloaders['val'], train=False)
        print(f"Epoch {epoch}/{max_epochs}, train_loss: {train_loss:.3f}, train_rmse: {train_rmse:.3f}, val_loss: {val_loss:.3f}, val_rmse: {val_rmse:.3f}")
        
        writer.add_scalar('train_loss', train_loss, epoch)
        writer.add_scalar('train_rmse', train_rmse, epoch)
        writer.add_scalar('val_loss', val_loss, epoch)
        writer.add_scalar('val_rmse', val_rmse, epoch)
        
        # Save best weights
        if val_rmse < best_val_rmse:
            best_epoch = epoch
            best_val_rmse = val_rmse
            best_model_weights = copy.deepcopy(model.state_dict())
            
        # Early stopping
        print(f"best val_rmse: {best_val_rmse:.3f}, epoch: {epoch}, best_epoch: {best_epoch}, current_patience: {patience - (epoch - best_epoch)}")
        if epoch - best_epoch >= patience:
            break
        
    torch.save(best_model_weights, f'/home/alexrichard/LRZ Sync+Share/ML in Physics/{NAME}.pth')

In [20]:
model = TracNet(n_channels=1).double()
print(summary(model, verbose=2))
model.to(device)
model.apply(initialize_weight)

inputs, targets = next(iter(dataloaders['train']))
writer.add_graph(model, inputs)

optimizer = torch.optim.SGD(model.parameters(), lr=0.0006, momentum=0.9, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=15, gamma=0.7943, verbose=True)

fit(model, optimizer, scheduler, dataloaders, max_epochs=5, patience=5)

Layer (type:depth-idx)                   Param #
TracNet                                  --
├─ConvBlock_1: 1-1                       --
│    └─conv_block_1.0.weight             ├─576
│    └─conv_block_1.1.weight             ├─32
│    └─conv_block_1.1.bias               └─32
│    └─Sequential: 2-1                   --
│    │    └─0.weight                     ├─576
│    │    └─1.weight                     ├─32
│    │    └─1.bias                       └─32
│    │    └─Conv3d: 3-1                  576
│    │    │    └─weight                  └─576
│    │    └─BatchNorm3d: 3-2             64
│    │    │    └─weight                  ├─32
│    │    │    └─bias                    └─32
│    │    └─ReLU: 3-3                    --
├─ConvBlock_2: 1-2                       --
│    └─conv_block_2.0.weight             ├─36,864
│    └─conv_block_2.0.bias               └─64
│    └─Sequential: 2-2                   --
│    │    └─0.weight                     ├─36,864
│    │    └─0.bias                 

/home/alexrichard/.local/lib/python3.8/site-packages/torch/nn/modules/conv.py:587: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:744.)
  return F.conv3d(


Adjusting learning rate of group 0 to 6.0000e-04.
Adjusting learning rate of group 0 to 6.0000e-04.
Epoch 1/5, train_loss: 255.790, train_rmse: 22.618, val_loss: 350.878, val_rmse: 26.491
best val_rmse: 26.491, epoch: 1, best_epoch: 1, current_patience: 5
Adjusting learning rate of group 0 to 6.0000e-04.
Epoch 2/5, train_loss: 250.575, train_rmse: 22.386, val_loss: 350.755, val_rmse: 26.486
best val_rmse: 26.486, epoch: 2, best_epoch: 2, current_patience: 5
Adjusting learning rate of group 0 to 6.0000e-04.
Epoch 3/5, train_loss: 241.538, train_rmse: 21.979, val_loss: 350.650, val_rmse: 26.482
best val_rmse: 26.482, epoch: 3, best_epoch: 3, current_patience: 5
Adjusting learning rate of group 0 to 6.0000e-04.
Epoch 4/5, train_loss: 230.497, train_rmse: 21.471, val_loss: 350.559, val_rmse: 26.479
best val_rmse: 26.479, epoch: 4, best_epoch: 4, current_patience: 5
Adjusting learning rate of group 0 to 6.0000e-04.
Epoch 5/5, train_loss: 219.430, train_rmse: 20.949, val_loss: 350.471, val_r

In [21]:
model.load_state_dict(torch.load(f'/home/alexrichard/LRZ Sync+Share/ML in Physics/{NAME}.pth'))
model.eval()

TracNet(
  (s1): ConvBlock_1(
    (conv_block_1): Sequential(
      (0): Conv3d(1, 32, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=same, bias=False)
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (s2): ConvBlock_2(
    (conv_block_2): Sequential(
      (0): Conv3d(32, 64, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=same)
      (1): ReLU(inplace=True)
    )
  )
  (s3): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (s4): ConvBlock_1(
    (conv_block_1): Sequential(
      (0): Conv3d(64, 64, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=same, bias=False)
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (s5): ConvBlock_2(
    (conv_block_2): Sequential(
      (0): Conv3d(64, 128, kernel_size=(2, 3, 3), stride=(1, 1, 1), padding=same)
      (1): ReLU(inplace=True)


## Evaluation

Load an exemplary pair of displacement and traction field from the test set

In [22]:
# Test data for working on Martinsried machine
sample_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/test/generic/testData104/dspl/MLData0022.mat'
target_path = '/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/test/generic/testData104/trac/MLData0022.mat'
sample = loadmat(sample_path)
target = loadmat(target_path)
# To numpy arrays
dspl_field = np.array(sample['dspl'])[np.newaxis, :]
gt_trac_field = np.array(target['trac'])[np.newaxis, :]
# Reshape
dspl_field = np.moveaxis(dspl_field[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
gt_trac_field = np.moveaxis(gt_trac_field[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
# To torch tensors
dspl_field = torch.from_numpy(dspl_field).double()
gt_trac_field = torch.from_numpy(gt_trac_field).double()

Define function to perform predictions with torch model

In [23]:
def predict(features):
    with torch.no_grad():
        return model(features)

In [24]:
predicted_trac_field = predict(dspl_field)

In [25]:
predicted_trac_field.shape

torch.Size([1, 1, 2, 104, 104])

In [26]:
predicted_trac_field = torch.squeeze(predicted_trac_field)

In [27]:
predicted_trac_field.shape

torch.Size([2, 104, 104])

In [28]:
predicted_trac_field = torch.permute(predicted_trac_field, (2, 1, 0))

In [29]:
predicted_trac_field.shape

torch.Size([104, 104, 2])

In [30]:
predicted_trac_field_dict = {"trac": np.array(predicted_trac_field)}

In [31]:
predicted_trac_field_dict["trac"]

array([[[-0.03892724, -0.03873764],
        [-0.03882611, -0.03856356],
        [-0.03883287, -0.03856733],
        ...,
        [-0.03884956, -0.03857117],
        [-0.03885003, -0.03857323],
        [-0.03887555, -0.03839725]],

       [[-0.03896169, -0.03885144],
        [-0.03814291, -0.03826772],
        [-0.03815294, -0.03826608],
        ...,
        [-0.03813888, -0.03827612],
        [-0.03815203, -0.03827021],
        [-0.03820124, -0.03808545]],

       [[-0.03890845, -0.03883635],
        [-0.03808166, -0.03824124],
        [-0.0380916 , -0.03824516],
        ...,
        [-0.03808969, -0.03825293],
        [-0.03810644, -0.03825474],
        [-0.03820846, -0.03807743]],

       ...,

       [[-0.03892663, -0.03883772],
        [-0.03810428, -0.03824574],
        [-0.03812084, -0.03824756],
        ...,
        [-0.03809588, -0.03825051],
        [-0.03811818, -0.03824834],
        [-0.03818386, -0.03806935]],

       [[-0.03891886, -0.03884017],
        [-0.03811199, -0.03

In [32]:
savemat('/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/torch_pred.mat', predicted_trac_field_dict)

Load the prediction of the Matlab model

In [33]:
matlab_prediction = loadmat('/home/alexrichard/LRZ Sync+Share/ML in Physics/DL-TFM-main/pred.mat')

In [34]:
matlab_predicted_trac_field = np.array(matlab_prediction['ans'])

In [35]:
matlab_predicted_trac_field.shape

(104, 104, 2)

In [36]:
matlab_predicted_trac_field = np.array(matlab_prediction['ans'])[np.newaxis, :]
matlab_predicted_trac_field = np.moveaxis(matlab_predicted_trac_field[:, np.newaxis], [2, 3, 4], [-1, 3, 2])
matlab_predicted_trac_field = torch.from_numpy(matlab_predicted_trac_field).double()

In [37]:
def forward(predictions, target):
    loss = 0.5 * torch.sum(torch.pow(target - predictions, 2))
    return loss

In [38]:
forward(predicted_trac_field, gt_trac_field)

RuntimeError: The size of tensor a (104) must match the size of tensor b (2) at non-singleton dimension 4

In [ ]:
forward(matlab_predicted_trac_field, gt_trac_field)

In [ ]:
torch.allclose(predicted_trac_field,matlab_predicted_trac_field, atol=5)

In [ ]:
predicted_trac_field

In [ ]:
matlab_predicted_trac_field

## Visualize data

In [ ]:
import shapely as sh

In [ ]:
def plotDspl(sample):
    zipped = np.array(list(zip(sample['brdx'][0], sample['brdy'][0])))  # array with (x,y) pairs of cell border coordinates
    polygon = sh.geometry.Polygon(zipped)  # create polygon

    interior = np.zeros((sample['dspl'].shape[0], sample['dspl'].shape[1]), dtype=int)  # create all zero matrix
    for i in range(len(interior)):  # set all elements in interior matrix to 1 that actually lie within the cell
        for j in range(len(interior[i])):
            point = Point(i, j)
            if polygon.contains(point):
                interior[i][j] = 1

    # plot polygon using geopandas
    p = gpd.GeoSeries(polygon)
    p.plot()
    plt.show()

In [ ]:
plotDspl(sample)